In [263]:
#Creating KMeans Clustering Algorithm from Scratch
#First get a dataset
from sklearn.datasets import load_iris
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import numpy as np
import random as rd

In [264]:
iris = load_iris()
data = iris.data
target = iris.target

In [265]:
XFull = data[:, 1:3]
XTrain, XTest, yTrain, yTest = train_test_split(XFull, target)

In [266]:
yTrain = yTrain.reshape(-1,1)
yTest = yTest.reshape(-1,1)
XTrain.shape, yTrain.shape

((112, 2), (112, 1))

My Implementation

In [289]:
#We need to define k Cluster centroids at the positions of k random data points
#Then we calculate each data point's distance from all k centroids and decide its label from the least distance

class KMeansClustering:
    n_clusters = 7
    threshold = 0.5
    data = []
    distances = []
    labels = []
    uniqueLabels = []
    centroidPos = []
    def __init__(self, n_clusters = 7, threshold = 1):
        self.n_clusters = n_clusters
        self.threshold = threshold
    
    def fit(self, data): 
        #Resetting any existing values
        self.data = np.array(data)
        self.distances = np.zeros(shape=self.data.shape)
        self.labels = np.zeros(shape=self.data.shape)
        self.uniqueLabels = []
        self.centroidPos = []
        self._cluster()

    def _cluster(self):
        self.uniqueLabels = [i for i in range(self.n_clusters)]
        centroidPos = []

        #Initializing centroids
        for i in range(self.n_clusters):
            options = np.array(self.data)
            pos = rd.choice(self.data)
            
            mask = options != pos
            options = options[mask]
            
            self.centroidPos.append(pos)
        self.centroidPos = np.array(self.centroidPos)

        converged = False
        centroidDist = 10000
        
        while(not converged):

            for i in range(len(self.data)):
                dist = []
                for cent in self.centroidPos:
                    dist.append(np.linalg.norm(self.data[i] - cent))
                min = np.min(dist)
                label = np.argmin(np.array(dist))
                self.distances[i] = min
                self.labels[i] = label
            
            #Now we need to change the postions of the centroids based on the mean of the labeled values
            #So loop over all the centroids, nested loop over all the data points and then find the mean where i == labels[j]
            
            newCentroidPos = []
            for i in range(len(self.centroidPos)):
                sum = 0
                count = 0
                for j in range(len(self.data)):
                    b = (self.labels[j] == i).astype(int)
                    sum += b * self.data[j]
                    count += 1
                newCentroidPos.append(sum/count)
                
            newCentroidPos = np.array(newCentroidPos)
            
            newCentroidDist = np.linalg.norm(newCentroidPos - self.centroidPos)
            converged = abs(newCentroidDist - centroidDist) < self.threshold
            centroidDist = newCentroidDist

In [268]:
def test(outputs):
    res = {"T": 0, "F": 0}
    for output in outputs:
        if(output):
            res["T"] += 1
        else: 
            res["F"] += 1
    print(res)

In [299]:
kmeans = KMeansClustering(n_clusters=3, threshold=0.0001)
kmeans.fit(XTrain)
yPred = kmeans.labels[:, 0].astype(int)

#Comparing to actual values
test(yPred)

5.34574889515024
5.34574889515024
{'T': 72, 'F': 40}


Testing Sklearn's implementation

In [270]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(XTrain)          

c:\Users\unova\anaconda3\envs\nb\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3)

In [271]:
yPred = kmeans.labels_.reshape(-1,1)
outputs = yPred == yTrain
outputs.shape

test(outputs)

{'T': 106, 'F': 6}


KMeans is highly dependent on the initial 